In [6]:
import sqlalchemy as db
from geopy.geocoders import Nominatim

In [10]:
geolocator = Nominatim(user_agent="process_routes")

In [25]:
username = "root"
password = "password"
server = "localhost"
database = "airplane_crashes"

engine = db.create_engine(f"mysql+pymysql://{username}:{password}@{server}/{database}")
metadata = db.MetaData()

crashes_tbl = db.Table('Crashes', metadata, autoload_with=engine)
routes_tbl = db.Table('Routes', metadata, autoload_with=engine)

In [27]:
with engine.connect() as con:
    for cid, route in con.execute(db.select(crashes_tbl.c.cid, crashes_tbl.c.route)):
        if route is None:
            continue
        
        for index, location in enumerate(route.replace(' -', ' - ').replace('- ', ' - ').split('-')):
            location = location.strip()
            
            con.execute(routes_tbl.insert().values(cid=cid, ordinal=index, location=location if location != '' else None))
            